In [1]:
import os
import argparse
from ml_collections import ConfigDict
import yaml
import time

import copy
import numpy as np
import torch
from torch import optim
from torch.utils.data import DataLoader
from torch_geometric.transforms import Compose
from tqdm import tqdm
import wandb
import numpy as np
import random

from data.data_preprocess import HeteroAddLaplacianEigenvectorPE, SubSample, SubSample_pad
from data.dataset import LPDataset
from data.utils import args_set_bool, collate_fn_ip
from models.hetero_gnn import TripartiteHeteroGNN, TripartiteHeteroGNN_, TripartiteHeteroGNN_new, BipartiteHeteroGNN
from trainer import Trainer

In [2]:
if wandb.run is not None:
    print('wandb running')
else:
    print('wandb not running')

wandb not running


In [3]:
# 2*ipm_steps/num_conv_layers = outer loop
# num_conv_layers = inner loop
var_dict = {
            "weight_decay": 0,
            "micro_batch": 8,         # oringal: 4, 
            "batchsize": 64,          # oringal: 128,
            "hidden": 180, 
            "num_conv_layers": 2,     # oringinal: 8,  
            "num_pred_layers": 4, 
            "num_mlp_layers": 4, 
#             "share_lin_weight": 'false', 
            "share_lin_weight": True, 
#             "conv_sequence": 'ceov', 
            "conv_sequence": 'cov', 
            "loss_weight_x": 1.0, 
            "loss_weight_obj": 3.43, 
            "loss_weight_cons": 5.8,     # oringinal: 5.8
            "losstype": 'l2',
            "runs": 3,
            "lappe": 0, 
            "conv": 'gcnconv', 
            "epoch": 50,    # oringal: 500
#             "ipm_alpha": 0.35,
            "ipm_alpha": 0.7,
    
            "ipm_steps": 16,       # 8 for supervising every mlp layer, 1 for supervising last layer
    
            "dropout": 0,
#             "share_conv_weight": 'false',        # default: false   
            "share_conv_weight": True,        # default: false  
#             "use_norm": 'true',
#             "use_res": 'false',  
            "use_norm": True,
            "use_res": True,  
            "lr": 1.e-5,     # original: 1.e-3
            "weight_decay": 0
            }

In [4]:
# wandb.init(project='reallocation_harp',
#            config=var_dict,
#            entity="fzhou")

In [5]:
if wandb.run is not None:
    print('wandb running')
else:
    print('wandb not running')

wandb not running


In [6]:
# -Ax+b<=-1
# Use TripartitionGNN

trainleq1_ins = 'trainleq1_abilene' 
validleq1_ins = 'validleq1_abilene'
testleq1_ins = 'testleq1_abilene'
ipm = 16

testleq1_dataset = LPDataset('raw',
                    extra_path=f'{1}restarts_'
                                     f'{0}lap_'
                                     f'{ipm}steps'
                                     f'{"_upper_" + str(testleq1_ins)}',
                    upper_bound=1,
                    rand_starts=1,
                    pre_transform=Compose([HeteroAddLaplacianEigenvectorPE(k=0),
                                                 SubSample(ipm)]))

test_loader = DataLoader(testleq1_dataset,
                        batch_size=var_dict['batchsize'],
                        shuffle=False,
                        num_workers=1,
                        collate_fn=collate_fn_ip)

In [7]:
len(testleq1_dataset)/var_dict["batchsize"], len(testleq1_dataset)%var_dict["batchsize"]

(31.5, 32)

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [9]:
#### test exact number of iteration model
# with sc
seed = 2026      # 2025, 2026, 2027, 2028 
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
np.random.seed(seed)  # Numpy module.
random.seed(seed)  # Python random module.
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

path = 'logs/o16i2_train_abilene_leq1_2026/run0/43_best_model.pt'
path = 'logs/o16i2_train_abilene_leq1_2026/run0/49_model.pt'


model = TripartiteHeteroGNN_(ipm_steps=var_dict['ipm_steps'],
                                conv=var_dict['conv'],
                                in_shape=2,
                                pe_dim=var_dict['lappe'],
                                hid_dim=var_dict['hidden'],
                                num_conv_layers=var_dict['num_conv_layers'],
                                num_pred_layers=var_dict['num_pred_layers'],
                                num_mlp_layers=var_dict['num_mlp_layers'],
                                dropout=var_dict['dropout'],
                                share_conv_weight=var_dict['share_conv_weight'],
                                share_lin_weight=var_dict['share_lin_weight'],
                                use_norm=var_dict['use_norm'],
                                use_res=var_dict['use_res'],
                                conv_sequence=var_dict['conv_sequence']).to(device)


trainer = Trainer(device,
                  'primal+objgap+constraint',
                  var_dict['losstype'],
                  var_dict['micro_batch'],
                  var_dict['ipm_steps'],
                  var_dict['ipm_alpha'],
                  loss_weight={'primal': var_dict['loss_weight_x'],
                               'objgap': var_dict['loss_weight_obj'],
                               'constraint': var_dict['loss_weight_cons']})

model.load_state_dict(torch.load(path))

num_pair = 132     # abilene (12*11, also the number of the first set of constraints Ax=1)
k = 8              # k-shortest path
edges_num = 30     # number of edges of abilene, also the number of the second constraint

with torch.no_grad():

    objs_gap, cons_gap, objs_nocgap, objs_norm_mlu, objs_mlu, norm_mlus, mlus = trainer.eval_metrics_harpleq1(k, num_pair, edges_num, var_dict['batchsize'], test_loader, model)

test_objgap = objs_gap[:, -1].mean()
test_consgap = cons_gap[:, -1].mean()
test_objgap_nocon = objs_nocgap[:, -1].mean()
test_obj_norm_mlu = objs_norm_mlu[:, -1].mean()
test_obj_mlu = objs_mlu[:, -1].mean()

test_norm_mlu = norm_mlus.mean()
test_mlu = mlus.mean()

print(test_objgap, test_consgap, test_objgap_nocon, test_obj_norm_mlu, test_obj_mlu, test_norm_mlu, test_mlu)

MLU tensor([0.5010, 0.4952, 0.4018, 0.4722, 0.5265, 1.5516, 4.8330, 3.2116, 4.1153,
        3.6278, 0.4892, 0.5469, 0.5615, 0.5479, 0.4466, 0.4750, 0.4757, 0.4854,
        0.4699, 0.5316, 0.5116, 0.5109, 0.5238, 0.5337, 0.5308, 0.5238, 0.5305,
        0.5363, 0.5273, 0.4643, 0.5018, 0.5070, 0.5429, 0.5166, 0.4336, 0.5182,
        0.5023, 0.5041, 0.5314, 0.5619, 0.5556, 0.5199, 0.5546, 0.5843, 0.4858,
        0.4982, 0.4829, 0.4801, 0.4813, 0.4787, 0.4866, 0.4832, 0.4506, 0.4531,
        0.4754, 0.4683, 0.4432, 0.4361, 0.4692, 0.4844, 0.4688, 0.4140, 0.3973,
        0.8131])
OPT tensor([0.3501, 0.3500, 0.3292, 0.3380, 0.3406, 1.0212, 1.3875, 0.9964, 1.3875,
        1.0388, 0.3416, 0.3367, 0.3415, 0.3416, 0.3462, 0.3525, 0.3440, 0.3392,
        0.3289, 0.3373, 0.3407, 0.3567, 0.3466, 0.3402, 0.3489, 0.3638, 0.3443,
        0.3441, 0.3475, 0.3418, 0.3309, 0.3239, 0.3104, 0.3122, 0.3169, 0.3162,
        0.3169, 0.3426, 0.3438, 0.3458, 0.3458, 0.3388, 0.3466, 0.3654, 0.3384,
        0.3459,

MLU tensor([0.5384, 0.5311, 0.5183, 0.5260, 0.5315, 0.5468, 0.5542, 0.5347, 0.5689,
        0.5377, 0.5297, 0.5200, 0.5338, 0.5368, 0.5643, 0.6666, 0.5548, 0.5786,
        0.5262, 0.5195, 0.5106, 0.5278, 0.5271, 0.4998, 0.4677, 0.5040, 0.4762,
        0.5204, 0.5025, 0.4978, 0.5068, 0.4688, 0.4406, 0.5119, 0.5165, 0.5263,
        0.4925, 0.5023, 0.4604, 0.5048, 0.4937, 0.5150, 0.5365, 0.5333, 0.5082,
        0.4922, 0.4954, 0.5317, 0.5224, 0.5109, 0.4747, 0.5138, 0.5138, 0.4851,
        0.4753, 0.4809, 0.5001, 0.5105, 0.4775, 0.4866, 0.5127, 0.4567, 0.4471,
        0.7199])
OPT tensor([0.2726, 0.2756, 0.2727, 0.2693, 0.2625, 0.3101, 0.2894, 0.2912, 0.2849,
        0.2848, 0.2859, 0.3028, 0.2940, 0.2943, 0.2952, 0.2915, 0.2769, 0.2776,
        0.2741, 0.2824, 0.2909, 0.2716, 0.2708, 0.2746, 0.2638, 0.2699, 0.2745,
        0.2700, 0.2608, 0.2713, 0.2738, 0.2539, 0.2535, 0.2731, 0.2712, 0.2745,
        0.2595, 0.2645, 0.2665, 0.2542, 0.2539, 0.2687, 0.2805, 0.2801, 0.2789,
        0.2830,

MLU tensor([0.5694, 0.5467, 0.5124, 0.5523, 0.4792, 0.5002, 0.5182, 0.5459, 0.5254,
        0.5855, 0.6033, 0.5924, 0.5681, 0.5359, 0.5330, 0.5204, 0.5022, 0.5329,
        0.4975, 0.5452, 0.5140, 0.5864, 0.5906, 0.5968, 0.6509, 0.6055, 0.5681,
        0.6413, 0.5913, 0.6004, 0.6451, 0.6191, 0.5945, 0.6203, 0.5855, 0.5628,
        0.5619, 0.5345, 0.5364, 0.5807, 0.5099, 0.5352, 0.5577, 0.5772, 0.5370,
        0.5827, 0.5541, 0.5433, 0.5377, 0.5147, 0.5234, 0.5815, 0.5097, 0.4930,
        0.5336, 0.5655, 0.5518, 0.5768, 0.5432, 0.5732, 0.5402, 0.5792, 0.5525,
        0.9563])
OPT tensor([0.3007, 0.2923, 0.2891, 0.3186, 0.3233, 0.3255, 0.3270, 0.3141, 0.3088,
        0.3272, 0.3256, 0.3253, 0.3311, 0.3327, 0.3314, 0.3117, 0.3120, 0.3284,
        0.3384, 0.3163, 0.3179, 0.3423, 0.3337, 0.3397, 0.3418, 0.3210, 0.3123,
        0.3297, 0.3299, 0.3215, 0.3215, 0.3160, 0.3138, 0.3137, 0.3199, 0.3199,
        0.3154, 0.2993, 0.3090, 0.3010, 0.3013, 0.3018, 0.3228, 0.3106, 0.3009,
        0.2948,

MLU tensor([0.5995, 0.5624, 0.5650, 0.5435, 0.5583, 0.5542, 0.5601, 0.6055, 0.5401,
        0.5644, 0.5203, 0.5365, 0.5615, 0.5519, 0.5296, 0.5571, 0.5269, 0.5540,
        0.5673, 0.5910, 0.5797, 0.5044, 0.5339, 0.5487, 0.5112, 0.5641, 0.5584,
        0.5680, 0.5545, 0.5641, 0.5110, 0.5752, 0.5681, 0.5741, 0.5534, 0.5593,
        0.5223, 0.5168, 0.5896, 0.5889, 0.5472, 0.5985, 0.5618, 0.6038, 0.5646,
        0.5455, 0.5622, 0.5562, 0.5377, 0.5153, 0.5392, 0.5177, 0.5624, 0.5986,
        0.5313, 0.5557, 0.4959, 0.5284, 0.5178, 0.5892, 0.5795, 0.6180, 0.5648,
        0.9582])
OPT tensor([0.3273, 0.3390, 0.3518, 0.3523, 0.3475, 0.3407, 0.3327, 0.3303, 0.3370,
        0.3476, 0.3415, 0.3279, 0.3353, 0.3343, 0.3320, 0.3214, 0.3254, 0.3341,
        0.3442, 0.3335, 0.3326, 0.3335, 0.3296, 0.3183, 0.3048, 0.3399, 0.3450,
        0.3257, 0.3155, 0.3163, 0.3336, 0.3192, 0.3309, 0.3502, 0.3513, 0.3287,
        0.3200, 0.3166, 0.3032, 0.3059, 0.3233, 0.3080, 0.3206, 0.3167, 0.3143,
        0.3244,

MLU tensor([0.4868, 0.4719, 0.5138, 0.5301, 0.5128, 0.5258, 0.5793, 0.5121, 0.4917,
        0.5408, 0.5501, 0.5588, 0.5546, 0.6052, 0.5288, 0.5222, 0.5145, 0.4762,
        0.4676, 0.4665, 0.4899, 0.4802, 0.4543, 0.5258, 0.4923, 0.5560, 0.5478,
        0.5531, 0.5352, 0.5678, 0.5317, 0.5669, 0.5505, 0.5661, 0.5336, 0.6419,
        0.5631, 0.5973, 0.5526, 0.5385, 1.5538, 0.5160, 0.5526, 0.5599, 0.5457,
        0.3744, 0.4124, 0.6620, 0.6279, 0.6983, 0.6880, 0.6918, 0.6545, 0.6641,
        1.5030, 2.0172, 1.8049, 0.4597, 0.6598, 1.3086, 0.7560, 0.7020, 0.7536,
        1.1923])
OPT tensor([0.3473, 0.3564, 0.3520, 0.3585, 0.3375, 0.3451, 0.3605, 0.3353, 0.3382,
        0.3163, 0.3230, 0.3396, 0.3500, 0.3562, 0.3833, 0.3536, 0.3333, 0.3449,
        0.3600, 0.3587, 0.3552, 0.3577, 0.3507, 0.3768, 0.3772, 0.3760, 0.3745,
        0.3783, 0.3750, 0.3759, 0.3640, 0.3622, 0.3774, 0.3789, 0.3926, 0.4051,
        0.3666, 0.3884, 0.4003, 0.4246, 1.0192, 0.4883, 0.4139, 0.4270, 0.4441,
        0.6315,

MLU tensor([0.5860, 0.5691, 0.5818, 0.5666, 0.5626, 0.5473, 0.5862, 0.5990, 0.6022,
        0.5726, 0.5524, 0.5653, 0.5888, 0.6779, 0.6409, 0.8117, 0.6557, 0.7011,
        0.6114, 0.6741, 0.6755, 0.6994, 0.6782, 0.6622, 0.6536, 0.7051, 0.7917,
        0.6838, 0.6158, 0.6230, 0.6280, 0.6972, 0.7411, 0.7345, 0.7827, 0.6671,
        0.7379, 0.8410, 0.7885, 0.8100, 0.7987, 0.7740, 0.6745, 0.7041, 0.6924,
        0.7763, 0.7296, 0.7020, 0.7570, 0.7408, 0.7455, 0.7490, 0.7508, 0.8316,
        0.7147, 0.7635, 0.7155, 0.6860, 0.7198, 0.7081, 0.6530, 0.6725, 0.6388,
        1.3056])
OPT tensor([0.3593, 0.3572, 0.3696, 0.3917, 0.3811, 0.3907, 0.3966, 0.3994, 0.3907,
        0.3979, 0.3882, 0.3871, 0.3953, 0.4085, 0.4127, 0.4122, 0.4128, 0.4175,
        0.4058, 0.4106, 0.4185, 0.4298, 0.4187, 0.4173, 0.4239, 0.4467, 0.4303,
        0.4446, 0.4390, 0.4300, 0.4168, 0.4235, 0.4160, 0.4159, 0.4155, 0.4199,
        0.4326, 0.4338, 0.4404, 0.4450, 0.4477, 0.4338, 0.4221, 0.4270, 0.4099,
        0.4250,

MLU tensor([0.6715, 0.7013, 0.7187, 0.7213, 0.6681, 0.6633, 0.7063, 0.6742, 0.6466,
        0.6971, 0.6534, 0.7011, 0.6562, 0.6508, 0.6927, 0.6966, 0.6246, 0.6242,
        0.6076, 0.6190, 0.5609, 0.6468, 0.6309, 0.6290, 0.7147, 0.7016, 0.6463,
        0.7073, 0.6627, 0.6529, 0.6708, 0.7045, 0.6552, 0.6426, 0.6410, 0.6133,
        0.6287, 0.6654, 0.6220, 0.6550, 0.6060, 0.6134, 0.6320, 0.6269, 0.6270,
        0.6393, 0.6420, 0.6451, 0.5995, 0.6605, 0.6214, 0.5805, 0.5812, 0.6109,
        0.6413, 0.6461, 0.6139, 0.6325, 0.6242, 0.6911, 0.6614, 0.6611, 0.6963,
        1.1035])
OPT tensor([0.4394, 0.4445, 0.4525, 0.4393, 0.4427, 0.4446, 0.4676, 0.4604, 0.4602,
        0.4555, 0.4480, 0.4513, 0.4438, 0.4373, 0.4501, 0.4345, 0.4278, 0.4384,
        0.4487, 0.4272, 0.4323, 0.4344, 0.4404, 0.4457, 0.4633, 0.4716, 0.4815,
        0.4654, 0.4502, 0.4448, 0.4330, 0.4378, 0.4453, 0.4528, 0.4532, 0.4456,
        0.4231, 0.4285, 0.4205, 0.4305, 0.4443, 0.4430, 0.4314, 0.4510, 0.4518,
        0.4440,

In [13]:
objs_gap[:, -1].shape, cons_gap[:, -1].shape, objs_nocgap[:, -1].shape

((2016,), (326592,), (2016,))

In [15]:
objs_norm_mlu[:, -1].shape, objs_mlu[:, -1].shape, norm_mlus.shape, mlus.shape

((2016,), (2016,), (2016,), (2016,))

In [28]:
cap = 0
dmd = 0
cap_gap = []
dmd_gap = []
for i in range(2016):
    idx = np.where(cons_gap[:, -1][i*162:(i+1)*162]!=0)[0]
    idx_ = np.where(idx>=132)[0]
#     print(cons_gap[:, -1][i*162:(i+1)*162][idx])
#     print(idx)
#     print(idx_)
    cap+= idx_.shape[0]
    dmd += (idx.shape[0] - idx_.shape[0])
#     print(idx_.shape, idx.shape)
    print(i*162, (i+1)*162-30, (i+1)*162)
    cap_gap.append(np.sum(cons_gap[:, -1][i*162-30:(i+1)*162]))
    dmd_gap.append(np.sum(cons_gap[:, -1][i*162:(i+1)*162-30]))

0 132 162
162 294 324
324 456 486
486 618 648
648 780 810
810 942 972
972 1104 1134
1134 1266 1296
1296 1428 1458
1458 1590 1620
1620 1752 1782
1782 1914 1944
1944 2076 2106
2106 2238 2268
2268 2400 2430
2430 2562 2592
2592 2724 2754
2754 2886 2916
2916 3048 3078
3078 3210 3240
3240 3372 3402
3402 3534 3564
3564 3696 3726
3726 3858 3888
3888 4020 4050
4050 4182 4212
4212 4344 4374
4374 4506 4536
4536 4668 4698
4698 4830 4860
4860 4992 5022
5022 5154 5184
5184 5316 5346
5346 5478 5508
5508 5640 5670
5670 5802 5832
5832 5964 5994
5994 6126 6156
6156 6288 6318
6318 6450 6480
6480 6612 6642
6642 6774 6804
6804 6936 6966
6966 7098 7128
7128 7260 7290
7290 7422 7452
7452 7584 7614
7614 7746 7776
7776 7908 7938
7938 8070 8100
8100 8232 8262
8262 8394 8424
8424 8556 8586
8586 8718 8748
8748 8880 8910
8910 9042 9072
9072 9204 9234
9234 9366 9396
9396 9528 9558
9558 9690 9720
9720 9852 9882
9882 10014 10044
10044 10176 10206
10206 10338 10368
10368 10500 10530
10530 10662 10692
10692 10824 10854

In [23]:
cap, dmd

(12528, 27834)

In [31]:
np.mean(cap_gap), np.sum(cap_gap)

(9.06296, 18270.926)

In [32]:
np.mean(dmd_gap), np.sum(dmd_gap)

(6.1950817, 12489.285)

In [ ]:
# harp average loss: 1.008026 

In [9]:
#### test exact number of iteration model
# with sc
seed = 2026      # 2025, 2026, 2027, 2028 
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
np.random.seed(seed)  # Numpy module.
random.seed(seed)  # Python random module.
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

path = 'logs/o16i2_train_abilene_leq1_2026/run0/43_best_model.pt'
path = 'logs/o16i2_train_abilene_leq1_2026/run0/49_model.pt'


model = TripartiteHeteroGNN_(ipm_steps=var_dict['ipm_steps'],
                                conv=var_dict['conv'],
                                in_shape=2,
                                pe_dim=var_dict['lappe'],
                                hid_dim=var_dict['hidden'],
                                num_conv_layers=var_dict['num_conv_layers'],
                                num_pred_layers=var_dict['num_pred_layers'],
                                num_mlp_layers=var_dict['num_mlp_layers'],
                                dropout=var_dict['dropout'],
                                share_conv_weight=var_dict['share_conv_weight'],
                                share_lin_weight=var_dict['share_lin_weight'],
                                use_norm=var_dict['use_norm'],
                                use_res=var_dict['use_res'],
                                conv_sequence=var_dict['conv_sequence']).to(device)


trainer = Trainer(device,
                  'primal+objgap+constraint',
                  var_dict['losstype'],
                  var_dict['micro_batch'],
                  var_dict['ipm_steps'],
                  var_dict['ipm_alpha'],
                  loss_weight={'primal': var_dict['loss_weight_x'],
                               'objgap': var_dict['loss_weight_obj'],
                               'constraint': var_dict['loss_weight_cons']})

model.load_state_dict(torch.load(path))

with torch.no_grad():

    train_losses, primal_losses, obj_losses, cons_losses = trainer.test_loss(test_loader, model)


self.ranked_convs ModuleList(
  (0): ModuleDict(
    (vals__to__cons): GCNConv()
    (obj__to__cons): GCNConv()
  )
  (1): ModuleDict(
    (vals__to__obj): GCNConv()
    (cons__to__obj): GCNConv()
  )
  (2): ModuleDict(
    (cons__to__vals): GCNConv()
    (obj__to__vals): GCNConv()
  )
)
cur_rank, cur_convs 0 ModuleDict(
  (vals__to__cons): GCNConv()
  (obj__to__cons): GCNConv()
) 2
-----------------------------
src and dst: vals cons
x_dict[src] torch.Size([67648, 360])
x_dict[dst] torch.Size([10368, 360])
con_econ 1
out_dict.keys() dict_keys(['cons'])
1 torch.Size([10368, 180])
-----------------------------
-----------------------------
src and dst: obj cons
x_dict[src] torch.Size([64, 360])
x_dict[dst] torch.Size([10368, 360])
con_econ 2
out_dict.keys() dict_keys(['cons'])
2 torch.Size([10368, 180])
-----------------------------
cur_rank, cur_convs 1 ModuleDict(
  (vals__to__obj): GCNConv()
  (cons__to__obj): GCNConv()
) 2
-----------------------------
src and dst: vals obj
x_dict[s

con_econ 1
out_dict.keys() dict_keys(['obj'])
1 torch.Size([64, 180])
-----------------------------
-----------------------------
src and dst: cons obj
x_dict[src] torch.Size([10368, 360])
x_dict[dst] torch.Size([64, 360])
con_econ 2
out_dict.keys() dict_keys(['obj'])
2 torch.Size([64, 180])
-----------------------------
cur_rank, cur_convs 2 ModuleDict(
  (cons__to__vals): GCNConv()
  (obj__to__vals): GCNConv()
) 2
-----------------------------
src and dst: cons vals
x_dict[src] torch.Size([10368, 360])
x_dict[dst] torch.Size([67648, 360])
con_econ 1
out_dict.keys() dict_keys(['vals'])
1 torch.Size([67648, 180])
-----------------------------
-----------------------------
src and dst: obj vals
x_dict[src] torch.Size([64, 360])
x_dict[dst] torch.Size([67648, 360])
con_econ 2
out_dict.keys() dict_keys(['vals'])
2 torch.Size([67648, 180])
-----------------------------
self.ranked_convs ModuleList(
  (0): ModuleDict(
    (vals__to__cons): GCNConv()
    (obj__to__cons): GCNConv()
  )
  (1):

self.ranked_convs ModuleList(
  (0): ModuleDict(
    (vals__to__cons): GCNConv()
    (obj__to__cons): GCNConv()
  )
  (1): ModuleDict(
    (vals__to__obj): GCNConv()
    (cons__to__obj): GCNConv()
  )
  (2): ModuleDict(
    (cons__to__vals): GCNConv()
    (obj__to__vals): GCNConv()
  )
)
cur_rank, cur_convs 0 ModuleDict(
  (vals__to__cons): GCNConv()
  (obj__to__cons): GCNConv()
) 2
-----------------------------
src and dst: vals cons
x_dict[src] torch.Size([67648, 360])
x_dict[dst] torch.Size([10368, 360])
con_econ 1
out_dict.keys() dict_keys(['cons'])
1 torch.Size([10368, 180])
-----------------------------
-----------------------------
src and dst: obj cons
x_dict[src] torch.Size([64, 360])
x_dict[dst] torch.Size([10368, 360])
con_econ 2
out_dict.keys() dict_keys(['cons'])
2 torch.Size([10368, 180])
-----------------------------
cur_rank, cur_convs 1 ModuleDict(
  (vals__to__obj): GCNConv()
  (cons__to__obj): GCNConv()
) 2
-----------------------------
src and dst: vals obj
x_dict[s

con_econ 1
out_dict.keys() dict_keys(['obj'])
1 torch.Size([64, 180])
-----------------------------
-----------------------------
src and dst: cons obj
x_dict[src] torch.Size([10368, 360])
x_dict[dst] torch.Size([64, 360])
con_econ 2
out_dict.keys() dict_keys(['obj'])
2 torch.Size([64, 180])
-----------------------------
cur_rank, cur_convs 2 ModuleDict(
  (cons__to__vals): GCNConv()
  (obj__to__vals): GCNConv()
) 2
-----------------------------
src and dst: cons vals
x_dict[src] torch.Size([10368, 360])
x_dict[dst] torch.Size([67648, 360])
con_econ 1
out_dict.keys() dict_keys(['vals'])
1 torch.Size([67648, 180])
-----------------------------
-----------------------------
src and dst: obj vals
x_dict[src] torch.Size([64, 360])
x_dict[dst] torch.Size([67648, 360])
con_econ 2
out_dict.keys() dict_keys(['vals'])
2 torch.Size([67648, 180])
-----------------------------
self.ranked_convs ModuleList(
  (0): ModuleDict(
    (vals__to__cons): GCNConv()
    (obj__to__cons): GCNConv()
  )
  (1):

con_econ 1
out_dict.keys() dict_keys(['vals'])
1 torch.Size([67648, 180])
-----------------------------
-----------------------------
src and dst: obj vals
x_dict[src] torch.Size([64, 360])
x_dict[dst] torch.Size([67648, 360])
con_econ 2
out_dict.keys() dict_keys(['vals'])
2 torch.Size([67648, 180])
-----------------------------
self.ranked_convs ModuleList(
  (0): ModuleDict(
    (vals__to__cons): GCNConv()
    (obj__to__cons): GCNConv()
  )
  (1): ModuleDict(
    (vals__to__obj): GCNConv()
    (cons__to__obj): GCNConv()
  )
  (2): ModuleDict(
    (cons__to__vals): GCNConv()
    (obj__to__vals): GCNConv()
  )
)
cur_rank, cur_convs 0 ModuleDict(
  (vals__to__cons): GCNConv()
  (obj__to__cons): GCNConv()
) 2
-----------------------------
src and dst: vals cons
x_dict[src] torch.Size([67648, 360])
x_dict[dst] torch.Size([10368, 360])
con_econ 1
out_dict.keys() dict_keys(['cons'])
1 torch.Size([10368, 180])
-----------------------------
-----------------------------
src and dst: obj cons


...

con_econ 1
out_dict.keys() dict_keys(['obj'])
1 torch.Size([64, 180])
-----------------------------
-----------------------------
src and dst: cons obj
x_dict[src] torch.Size([10368, 360])
x_dict[dst] torch.Size([64, 360])
con_econ 2
out_dict.keys() dict_keys(['obj'])
2 torch.Size([64, 180])
-----------------------------
cur_rank, cur_convs 2 ModuleDict(
  (cons__to__vals): GCNConv()
  (obj__to__vals): GCNConv()
) 2
-----------------------------
src and dst: cons vals
x_dict[src] torch.Size([10368, 360])
x_dict[dst] torch.Size([67648, 360])
con_econ 1
out_dict.keys() dict_keys(['vals'])
1 torch.Size([67648, 180])
-----------------------------
-----------------------------
src and dst: obj vals
x_dict[src] torch.Size([64, 360])
x_dict[dst] torch.Size([67648, 360])
con_econ 2
out_dict.keys() dict_keys(['vals'])
2 torch.Size([67648, 180])
-----------------------------
self.ranked_convs ModuleList(
  (0): ModuleDict(
    (vals__to__cons): GCNConv()
    (obj__to__cons): GCNConv()
  )
  (1):

con_econ 1
out_dict.keys() dict_keys(['cons'])
1 torch.Size([10368, 180])
-----------------------------
-----------------------------
src and dst: obj cons
x_dict[src] torch.Size([64, 360])
x_dict[dst] torch.Size([10368, 360])
con_econ 2
out_dict.keys() dict_keys(['cons'])
2 torch.Size([10368, 180])
-----------------------------
cur_rank, cur_convs 1 ModuleDict(
  (vals__to__obj): GCNConv()
  (cons__to__obj): GCNConv()
) 2
-----------------------------
src and dst: vals obj
x_dict[src] torch.Size([67648, 360])
x_dict[dst] torch.Size([64, 360])
con_econ 1
out_dict.keys() dict_keys(['obj'])
1 torch.Size([64, 180])
-----------------------------
-----------------------------
src and dst: cons obj
x_dict[src] torch.Size([10368, 360])
x_dict[dst] torch.Size([64, 360])
con_econ 2
out_dict.keys() dict_keys(['obj'])
2 torch.Size([64, 180])
-----------------------------
cur_rank, cur_convs 2 ModuleDict(
  (cons__to__vals): GCNConv()
  (obj__to__vals): GCNConv()
) 2
-----------------------------


con_econ 1
out_dict.keys() dict_keys(['obj'])
1 torch.Size([64, 180])
-----------------------------
-----------------------------
src and dst: cons obj
x_dict[src] torch.Size([10368, 360])
x_dict[dst] torch.Size([64, 360])
con_econ 2
out_dict.keys() dict_keys(['obj'])
2 torch.Size([64, 180])
-----------------------------
cur_rank, cur_convs 2 ModuleDict(
  (cons__to__vals): GCNConv()
  (obj__to__vals): GCNConv()
) 2
-----------------------------
src and dst: cons vals
x_dict[src] torch.Size([10368, 360])
x_dict[dst] torch.Size([67648, 360])
con_econ 1
out_dict.keys() dict_keys(['vals'])
1 torch.Size([67648, 180])
-----------------------------
-----------------------------
src and dst: obj vals
x_dict[src] torch.Size([64, 360])
x_dict[dst] torch.Size([67648, 360])
con_econ 2
out_dict.keys() dict_keys(['vals'])
2 torch.Size([67648, 180])
-----------------------------
self.ranked_convs ModuleList(
  (0): ModuleDict(
    (vals__to__cons): GCNConv()
    (obj__to__cons): GCNConv()
  )
  (1):

con_econ 1
out_dict.keys() dict_keys(['vals'])
1 torch.Size([67648, 180])
-----------------------------
-----------------------------
src and dst: obj vals
x_dict[src] torch.Size([64, 360])
x_dict[dst] torch.Size([67648, 360])
con_econ 2
out_dict.keys() dict_keys(['vals'])
2 torch.Size([67648, 180])
-----------------------------
self.ranked_convs ModuleList(
  (0): ModuleDict(
    (vals__to__cons): GCNConv()
    (obj__to__cons): GCNConv()
  )
  (1): ModuleDict(
    (vals__to__obj): GCNConv()
    (cons__to__obj): GCNConv()
  )
  (2): ModuleDict(
    (cons__to__vals): GCNConv()
    (obj__to__vals): GCNConv()
  )
)
cur_rank, cur_convs 0 ModuleDict(
  (vals__to__cons): GCNConv()
  (obj__to__cons): GCNConv()
) 2
-----------------------------
src and dst: vals cons
x_dict[src] torch.Size([67648, 360])
x_dict[dst] torch.Size([10368, 360])
con_econ 1
out_dict.keys() dict_keys(['cons'])
1 torch.Size([10368, 180])
-----------------------------
-----------------------------
src and dst: obj cons


con_econ 2
out_dict.keys() dict_keys(['obj'])
2 torch.Size([32, 180])
-----------------------------
cur_rank, cur_convs 2 ModuleDict(
  (cons__to__vals): GCNConv()
  (obj__to__vals): GCNConv()
) 2
-----------------------------
src and dst: cons vals
x_dict[src] torch.Size([5184, 360])
x_dict[dst] torch.Size([33824, 360])
con_econ 1
out_dict.keys() dict_keys(['vals'])
1 torch.Size([33824, 180])
-----------------------------
-----------------------------
src and dst: obj vals
x_dict[src] torch.Size([32, 360])
x_dict[dst] torch.Size([33824, 360])
con_econ 2
out_dict.keys() dict_keys(['vals'])
2 torch.Size([33824, 180])
-----------------------------
self.ranked_convs ModuleList(
  (0): ModuleDict(
    (vals__to__cons): GCNConv()
    (obj__to__cons): GCNConv()
  )
  (1): ModuleDict(
    (vals__to__obj): GCNConv()
    (cons__to__obj): GCNConv()
  )
  (2): ModuleDict(
    (cons__to__vals): GCNConv()
    (obj__to__vals): GCNConv()
  )
)
cur_rank, cur_convs 0 ModuleDict(
  (vals__to__cons): GCN

In [10]:
train_losses, primal_losses, obj_losses, cons_losses

(0.02859009447551909,
 0.0064540160080743215,
 0.0063992563694242445,
 3.2177468436577966e-05)